In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2019)

In [2]:
# trn = pd.read_csv('./dataset/train_ver2.csv')
trn = pd.read_csv('./dataset/train_delete_empty.csv')
tst = pd.read_csv('./dataset/test_ver2.csv')

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5,8,10,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Preprocessing

In [3]:
print('before tranin set shape : ', trn.shape)
prods = trn.columns[24:].tolist()

trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# # if some customer buy anyting, remove
# no_product = trn[prods].sum(axis=1) == 0
# trn = trn[~no_product]
# print('after tranin set shape : ', trn.shape)

before tranin set shape :  (11091070, 48)


In [4]:
for col in trn.columns[24:]:
    tst[col] = 0

df = pd.concat([trn, tst], axis=0)

In [5]:
features = []

# 범주형을 0, 1, 2와 같은 꼴로 변경시킴
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

In [6]:
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

In [7]:
df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

In [8]:
df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

In [9]:
df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

In [10]:
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

# Feature Engineering

In [11]:
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

In [12]:
df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [13]:
# lag_features = features.copy()

In [14]:
df.fillna(-99, inplace=True)

In [15]:
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    
    return int_date

In [16]:
# df['int_date'] = trn['fecha_dato'].map(date_to_int).astype(np.int8)
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [17]:
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

In [18]:
print(len(df_lag.columns))
print(len(df.columns))

53
53


In [19]:
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')
del df, df_lag

In [20]:
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

In [21]:
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

# Seperate validation set

In [22]:
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

In [23]:
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    #Extract new purchase
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype = np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

In [24]:
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

# Model training

In [25]:
param = {
    'booster' : 'gbtree',
    'max_depth' : 8,
    'nthread' : 8,
    'num_class' : len(prods),
    'objective' : 'multi:softprob',
    'silent' : 1,
    'eval_metric' : 'mlogloss',
    'eta' : 0.1,
    'min_child_weight' : 10,
    'colsample_bytree' : 0.8,
    'colsample_bylevel' : 0.9,
    'seed': 2019,
}

In [31]:
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [32]:
X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [33]:
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

[0]	train-mlogloss:2.74806	eval-mlogloss:2.75872
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:2.51359	eval-mlogloss:2.53049
[2]	train-mlogloss:2.32365	eval-mlogloss:2.34081
[3]	train-mlogloss:2.17123	eval-mlogloss:2.18886
[4]	train-mlogloss:2.05218	eval-mlogloss:2.07027
[5]	train-mlogloss:1.95326	eval-mlogloss:1.97175
[6]	train-mlogloss:1.86734	eval-mlogloss:1.88611
[7]	train-mlogloss:1.79414	eval-mlogloss:1.81337
[8]	train-mlogloss:1.7303	eval-mlogloss:1.74977
[9]	train-mlogloss:1.67556	eval-mlogloss:1.69546
[10]	train-mlogloss:1.63035	eval-mlogloss:1.65057
[11]	train-mlogloss:1.58563	eval-mlogloss:1.606
[12]	train-mlogloss:1.54697	eval-mlogloss:1.56784
[13]	train-mlogloss:1.51232	eval-mlogloss:1.53336
[14]	train-mlogloss:1.47952	eval-mlogloss:1.50094
[15]	train-mlogloss:1.44954	eval-mlogloss:1.47134
[16]	train-mlogloss:1.42299	eval-mlogloss:1.44501
[17]	train-m

[161]	train-mlogloss:1.00119	eval-mlogloss:1.08868
[162]	train-mlogloss:1.00058	eval-mlogloss:1.08862
[163]	train-mlogloss:0.999928	eval-mlogloss:1.08861
[164]	train-mlogloss:0.99934	eval-mlogloss:1.08858
[165]	train-mlogloss:0.998781	eval-mlogloss:1.08854
[166]	train-mlogloss:0.998275	eval-mlogloss:1.08853
[167]	train-mlogloss:0.997695	eval-mlogloss:1.08849
[168]	train-mlogloss:0.997152	eval-mlogloss:1.08849
[169]	train-mlogloss:0.996505	eval-mlogloss:1.08848
[170]	train-mlogloss:0.995766	eval-mlogloss:1.08838
[171]	train-mlogloss:0.995281	eval-mlogloss:1.08837
[172]	train-mlogloss:0.994813	eval-mlogloss:1.0883
[173]	train-mlogloss:0.994196	eval-mlogloss:1.08825
[174]	train-mlogloss:0.993532	eval-mlogloss:1.08822
[175]	train-mlogloss:0.992956	eval-mlogloss:1.08821
[176]	train-mlogloss:0.992307	eval-mlogloss:1.0882
[177]	train-mlogloss:0.991782	eval-mlogloss:1.08819
[178]	train-mlogloss:0.99119	eval-mlogloss:1.08815
[179]	train-mlogloss:0.990553	eval-mlogloss:1.08814
[180]	train-mloglo

# Save trained model

In [34]:
import pickle

pickle.dump(model, open("model/xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit
best_ntree_limit

187

# Validation

In [35]:
from mapk import *

In [36]:
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]
    
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [37]:
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

In [38]:
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

0.04266379915553903


In [39]:
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [40]:
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [41]:
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y, p, ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])

In [42]:
print(mapk(add_vld_list, result_vld, 7, 0.0))

0.03646330962404435


# Training Total Dataset

In [43]:
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [44]:
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))
print(best_ntree_limit)
print(model.best_ntree_limit)
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

230
187
[0]	train-mlogloss:2.74837
[1]	train-mlogloss:2.51447
[2]	train-mlogloss:2.32416
[3]	train-mlogloss:2.17134
[4]	train-mlogloss:2.05219
[5]	train-mlogloss:1.9533
[6]	train-mlogloss:1.86703
[7]	train-mlogloss:1.79361
[8]	train-mlogloss:1.7297
[9]	train-mlogloss:1.67508
[10]	train-mlogloss:1.62985
[11]	train-mlogloss:1.58494
[12]	train-mlogloss:1.54625
[13]	train-mlogloss:1.51151
[14]	train-mlogloss:1.47869
[15]	train-mlogloss:1.44879
[16]	train-mlogloss:1.4223
[17]	train-mlogloss:1.39707
[18]	train-mlogloss:1.37504
[19]	train-mlogloss:1.3538
[20]	train-mlogloss:1.33495
[21]	train-mlogloss:1.31817
[22]	train-mlogloss:1.30164
[23]	train-mlogloss:1.28707
[24]	train-mlogloss:1.2732
[25]	train-mlogloss:1.26107
[26]	train-mlogloss:1.24873
[27]	train-mlogloss:1.23765
[28]	train-mlogloss:1.22734
[29]	train-mlogloss:1.21746
[30]	train-mlogloss:1.20842
[31]	train-mlogloss:1.1999
[32]	train-mlogloss:1.19219
[33]	train-mlogloss:1.18437
[34]	train-mlogloss:1.1776
[35]	train-mlogloss:1.17083
[

In [45]:
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

Feature importance:
('renta', 19730)
('age', 18438)
('antiguedad', 17211)
('age_prev', 12567)
('antiguedad_prev', 11923)
('fecha_alta_month', 11172)
('nomprov', 10859)
('fecha_alta_year', 8926)
('renta_prev', 7657)
('canal_entrada', 7381)
('nomprov_prev', 6147)
('canal_entrada_prev', 4386)
('fecha_alta_month_prev', 4030)
('sexo', 3227)
('ind_recibo_ult1_prev', 3215)
('fecha_alta_year_prev', 3147)
('ind_ecue_fin_ult1_prev', 3022)
('ind_cco_fin_ult1_prev', 2908)
('ind_cno_fin_ult1_prev', 2709)
('segmento', 2175)
('ind_reca_fin_ult1_prev', 1998)
('ind_tjcr_fin_ult1_prev', 1994)
('segmento_prev', 1874)
('ind_nom_pens_ult1_prev', 1623)
('tiprel_1mes', 1545)
('ind_nomina_ult1_prev', 1522)
('ind_valo_fin_ult1_prev', 1506)
('ind_dela_fin_ult1_prev', 1436)
('ind_ctop_fin_ult1_prev', 1348)
('ind_actividad_cliente', 1252)
('sexo_prev', 1184)
('tiprel_1mes_prev', 1143)
('ind_ctpp_fin_ult1_prev', 1108)
('ind_fond_fin_ult1_prev', 982)
('ind_actividad_cliente_prev', 870)
('ind_ctma_fin_ult1_prev', 85

In [48]:
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [49]:
ncodpers_tst = tst.as_matrix(columns=['ncodpers'])
preds_tst = preds_tst - tst.as_matrix(columns=[prod + '_prev' for prod in prods])

E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\ProgramData\Anaconda3\envs\dsc\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


# Save Final model

In [50]:
pickle.dump(model, open("model/xgb.baseline_total.pkl", "wb"))

In [ ]:
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y, p, ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])
print(mapk(add_vld_list, result_vld, 7, 0.0))

# Create submit file

In [51]:
submit_file = open('model/xgb.baseline.2019-06-18_baseline2', 'w')
submit_file.write('ncodpers,added_products\n')
count = 0
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y, p, ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))
submit_file.close()